In [1]:
import numpy as np
import os
import pickle
import gc
import re
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import clear_output

from tensorflow.keras import layers, Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Layer
from tensorflow.python.keras.layers.convolutional import Conv3DTranspose
from tensorflow.python.ops.init_ops_v2 import he_normal
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

# from tensorflow.keras import mixed_precision
print("TensorFlow version: ", tf.__version__)

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow version:  2.10.0


In [2]:
def _clamp_disp(disp, min_disp, max_disp):
    """Clip max disparity, ortherwise it'll be hard for network to learn really big disparity/close object"""
    return np.clip(disp, min_disp, max_disp)

def histogram_equalization(image):
    # 이미지를 float32 타입으로 변환합니다.
    image = tf.cast(image, tf.float32)
    
    # 이미지를 [0, 1] 범위로 정규화합니다.
    image = tf.clip_by_value(image/255.0, 0.0, 1.0)
    
    # 이미지의 히스토그램을 계산합니다.
    hist = tf.histogram_fixed_width(image, [0.0, 1.0], nbins=256)
    
    # 누적 분포 함수(Cumulative Distribution Function)를 계산합니다.
    cdf = tf.cumsum(hist)
    cdf_min = cdf[tf.reduce_min(tf.where(tf.greater(cdf, 0)))]
    num_pixels = tf.reduce_sum(hist)
    cdf_normalized = (cdf - cdf_min) / (num_pixels - cdf_min)
    
    # 새로운 이미지를 만듭니다.
    image = tf.cast(tf.gather(cdf_normalized, tf.cast(image * 255, tf.int32)), tf.float32)
    
    return image

def StereoDataloader(img_left, img_right, disp_left, img_h, img_w, df_h, df_w, batch_num, ComPerBatch, data_ord, max_disp):
    tmp_img = []
    tmp_disp = []
    if df_h > 32 :
        randomH = np.random.randint(0, df_h)
    else :
        randomH = 0
    if df_w > 32 :
        randomW = np.random.randint(0, df_w)
    else :
        randomW = 0
    
    
    for idx in range(batch_num*ComPerBatch,(batch_num+1)*ComPerBatch):
        l = np.array(Image.open(img_left[data_ord[idx]]).convert("L"))[randomH:randomH+img_h, randomW:randomW+img_w]
        l = np.expand_dims(histogram_equalization(l), axis=2)        
        
        r = np.array(Image.open(img_right[data_ord[idx]]).convert("L"))[randomH:randomH+img_h, randomW:randomW+img_w]
        r = np.expand_dims(histogram_equalization(r), axis=2)        
        tmp_img.append(np.concatenate((l,r), axis=2))
        
        dispL = np.loadtxt(disp_left[data_ord[idx]], delimiter=",", dtype=np.float32)[randomH:randomH+img_h, randomW:randomW+img_w]
        tmp_disp.append(_clamp_disp(dispL,0,max_disp))
    
    return np.array(tmp_img, dtype=np.float32), np.array(tmp_disp, dtype=np.float32)

In [3]:
#### Loading dataset path
k_h = 4
k_w = 6

finl = open('gh_imgl.pkl','rb')
finr = open('gh_imgr.pkl', 'rb')
img_left = pickle.load(finl)
img_right = pickle.load(finr)
finl.close()
finr.close()

finl = open('gh_disp.pkl', 'rb')
disp_left = pickle.load(finl)
finl.close()

#training/test data set
tot_num1 = len(img_left)//2 # Half number, training data
tot_num2 = len(img_left) # total number, training+test data
file_max = tot_num2 # if training, file_max = tot_num1. Elif testing performance,file_max = tot_num2. 

# data size
max_w = 984
max_h = 560
max_disp = 160

img_w = 960
img_h = 544
df_w = max_w - img_w
df_h = max_h - img_h

feature_n = 32 # Default Feature Number of GC-Net
ComPerBatch = 1 #24 #256 # component number per batch
batch_size = np.int32(tot_num2/ComPerBatch) # number of batch
totpx_batch = img_w*img_h*ComPerBatch

# if you want to save data, save_data = 1,
#  else save_data = 0
save_data = 1 

# Image save function
def disp_img(img, title): 
    plt.figure(figsize=(5, 3))
    plt.title(title)
    plt.axis('off')
    plt.imshow(img, cmap='jet')
    plt.colorbar(shrink=0.5) 
    plt.tight_layout()
    plt.show()
    
    plt.close()
    gc.collect()

vmin = 40
vmax = 80
vgap = 10
# Image save function
def _save_img(img, title, dest, file_name, data_ind):
#     plt.switch_backend('Agg')
    plt.rc('font', size=30)        # 기본 폰트 크기
    plt.rcParams["font.family"] = "Times New Roman"
    
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.axis('off')
    ax = plt.gca()
    im = ax.imshow(img, cmap='jet', vmin=vmin, vmax=vmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_ticks(np.arange(vmin, vmax, vgap))  # 32 간격으로 눈금 설정    
    plt.tight_layout()
    # plt.show()
    plt.savefig('%s/%s_%05d.png' %(dest,file_name,data_ind))
    
    plt.close()    
    gc.collect()

In [4]:
# IUHM performance #############################################################################
max_epochs1 = 177

net = tf.keras.models.load_model('saved_model/IUHM_H%dWW%d_iter%d' %(k_h,k_w,max_epochs1))
dest_dir='IUHM_H%dWW%d_iter%d' %(k_h,k_w,max_epochs1)

if not os.path.isdir(dest_dir):
    os.makedirs(dest_dir)

mix_set = np.arange(file_max) #if training : mix_set = np.random.permutation(file_max)

file_name = '%s/test.csv' %(dest_dir)
f = open(file_name, 'w')    
f.write('Data No./Number as Disparity Difference, IUHM_H%dW%d_GH2470_epc%d loss, ratio of px<=1, <=2, <=3\n' %(k_h,k_w,max_epochs1))
f.close()

for batch_n in range(batch_size):
    DB_imgs,DB_disp = StereoDataloader(
        img_left = img_left,
        img_right = img_right,
        disp_left = disp_left,                
        img_h = img_h,
        img_w = img_w,
        df_h = df_h,
        df_w = df_w,
        batch_num = batch_n, # order of batch
        ComPerBatch = ComPerBatch, # component number of a batch
        data_ord = mix_set,
        max_disp = max_disp
    )

    # Model
    if save_data == 0:
        print('Batch %d/%d' %(batch_n+1,batch_size))
        result = net.predict(DB_imgs, batch_size=1, verbose=1)
        result = _clamp_disp(result,0, max_disp)
    elif save_data == 1:
        print('\rBatch %d/%d' %(batch_n+1,batch_size), end='')
        result = _clamp_disp(net.predict(DB_imgs, batch_size=1, verbose=0),0, max_disp)
        loss1 = np.mean(np.abs(DB_disp-result))

        file_name = '%s/test.csv' %(dest_dir)
        f = open(file_name, 'a')

        disp_diff1 = np.abs(DB_disp-result)

        f.write('%d, ' %(batch_n+1))

        # Model disparity error and px area ratio (<= 1px,2px,3px)        
        f.write('%.5f, ' %loss1)        
        f.write('%.5f, ' %(np.sum(disp_diff1<=1)/totpx_batch))        
        f.write('%.5f, ' %(np.sum(disp_diff1<=2)/totpx_batch))        
        f.write('%.5f \n'%(np.sum(disp_diff1<=3)/totpx_batch))

        f.close()

        #### Disparity calculated by Trueth Data            
        title = 'Ground Truth_Data#%d' %(batch_n+1)
        file_name = 'True_Disp'
        _save_img(DB_disp[0,:,:], title, dest_dir, file_name, batch_n+1)

        #### Disparity calculated by Stereo Camera Data with Model
        title = 'IUHM_H%dW%d_Data#%d' %(k_h,k_w,batch_n+1)
        file_name = 'Stereo_Disp'
        _save_img(result[0,:,:], title, dest_dir, file_name, batch_n+1)

        del file_name
        del loss1
        del disp_diff1
        gc.collect()

    del DB_imgs
    del DB_disp
    del result
    gc.collect()
    
del net

tf.keras.backend.clear_session()
gc.collect()

Batch 2470/2470

121252